In [ ]:
path = []


In [ ]:
path = [[0, 10], [1, 9], [2, 9], [3, 10], [4, 9], [5, 10], [6, 9], [7, 9], [8, 10], [9, 10], [10, 11], [11, 10], [12, 10], [13, 11],[14, 11], [15, 11], [16, 10], [17, 10], [18, 9], [19, 10]]

In [ ]:
import cv2
import numpy as np
blank = cv2.imread("blank.png",0)
blank = cv2.resize(blank,(20,20))
# path = np.array(path)
# epsilon = 0.1*cv2.arcLength(path, True)
# path = cv2.approxPolyDP(path, epsilon, False)
# print(path)
pnts = []
for i in range(1,16):
    pts = [np.mean(path[0:15*(i)][0]),np.mean(path[0:15*(i)][1])]
    pnts.append(pts)
#     cv2.line(blank,(path[i][1],path[i][0]),(path[i+1][1],path[i+1][0]),0)
#     pnts = np.mean(path[i:i+15][0])
print(pnts)
blank = cv2.resize(blank,(680,480))
cv2.imshow("blank",blank)
q = cv2.waitKey(1)
if q == 27:
    cv2.destroyAllWindows()

In [ ]:
import math
slp = []
for pix in pnts:
    slp.append(math.atan2(340-pix[1],pix[0]-240) - 1.57)

In [ ]:
import math
print(slp)

In [3]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
import numpy as np
import time
import threading

In [4]:
#output:: 6 category  
#input:: 15 slope 
#mode = xgboost
minibatch = []

In [ ]:
import random
from collections import deque
class DQNAgent:
    def __init__(self):
        self.state_size = 15
        self.action_size = 6
        self.memory = deque(maxlen=pow(10,6))
        self.gamma = 0.8
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.bst = xgb.Booster({'nthread':4})
        self.name = 'steerDeepQL'
        self.modelname = 'xgbmodel'
        self.load_bst()
    
    def xgbmodel_bld(self,data,label):
        param = {'base_score' : 1, 'max_depth': 7, 'eta': 0.1, #'updater':'refresh',
            #'process_type': 'update',
            'refresh_leaf': True,
            'reshape': True,
            'reg_alpha': 3, 
            'silent': 1, 
            'objective': 'multi:softprob',     
            'num_class': 6}  
        param['nthread'] = 4
        param['eval_metric'] = 'merror'
        num_round = 500  
        dtrain = xgb.DMatrix(data, label=label)
        evallist = [(dtrain, 'train'), (dtrain, 'eval')]
        self.bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=3, xgb_model = self.name)
        self.save_bst()

        
    def load_bst(self):
        self.bst.load_model(self.name)
    
    def save_bst(self):
        self.bst.save_model(self.name)
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.bst.predict(xgb.DMatrix(state))
#         print('act_values',act_values)
        return np.argmax(act_values[0])  
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        minibatch = self.memory
#         print(len(minibatch))
        bst_target = xgb.Booster({'nthread':4})
        bst_target.load_model(self.name)
        st = []
        tg = []
        for state, action, reward, next_state, done in minibatch:

            target = bst_target.predict(xgb.DMatrix(state))##target is Q matrix
            if done:
                target[0][action] = reward
            else:
                t = self.bst.predict(xgb.DMatrix(next_state))[0]
                target[0][action] = (1-self.gamma)*reward + self.gamma * np.amax(t)
            st.append(state)
            tg.append(np.argmax(target[0]))
        self.memory.clear()
        st = np.array(st)
        st = np.reshape(st, [-1, 15])
        tg = np.array(tg)
        tg = np.reshape(tg, [-1,1])
        print(st.shape, tg.shape)
        self.xgbmodel_bld(st, tg)


        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def decl_rew(self,state):
        rew = (np.sum((np.array([1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75]) - abs(state))*np.array([10,10,10,9,9,9,8,8,7,6,5,4,3,2,1])))/175
#         print('reward',rew)
#         print('state',state)
        done = False
        if rew > 130:
            done = True
        return rew,done
    
    def get_nextstate(self):
        ########
        stt = np.random.randint(175, size=15)/100
        return stt
    
def steer(GameOn):
    state_size = 15
    action_size = 6
    agent = DQNAgent()
    batch_size = 50
    state = np.zeros((15,), dtype=float)
    state = agent.get_nextstate()
#     agent.load_bst()
    
    while GameOn:
        start = time.time()
        state = np.reshape(state, [1, state_size])
        action = agent.act(state)
#         execute_action(action)
        reward, done = agent.decl_rew(state)
#         reward = reward 
        next_state = agent.get_nextstate()
        if(next_state == []):
            continue;
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done and len(agent.memory) >= 50 or len(agent.memory) >= 500 :
            print("replaying at {} xgboost".format(len(agent.memory)))
            t1 = threading.Thread(target=agent.replay, args=())
            t1.start()
        time.sleep(0.03)
#         print("FPS: ", 1.0 / (time.time() - start))
        

In [ ]:
steer(True)

/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:112: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


replaying at 500 xgboost
replaying at 501 xgboost
replaying at 502 xgboost
replaying at 503 xgboost
replaying at 504 xgboost
replaying at 505 xgboost


Exception in thread Thread-528:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-7d95b7becf25>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration

Exception in thread Thread-529:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-7d95b7becf25>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration



replaying at 506 xgboost
replaying at 507 xgboost
replaying at 508 xgboost
replaying at 509 xgboost
replaying at 510 xgboost
replaying at 511 xgboost


Exception in thread Thread-530:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-7d95b7becf25>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration



...


replaying at 1228 xgboost
replaying at 1229 xgboost
replaying at 1230 xgboost
replaying at 1231 xgboost
replaying at 1232 xgboost
replaying at 1233 xgboost
replaying at 1234 xgboost
replaying at 1235 xgboost
replaying at 1236 xgboost
replaying at 1237 xgboost


Too many open files
s = self._socket_class(self, socket_type, **kwargs)
  File "zmq/backend/cython/socket.pyx", line 330, in zmq.backend.cython.socket.Socket.__init__

zmq.error  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/zmq/sugar/socket.py", line 58, in __init__
.    ZMQErrorsuper(Socket, self).__init__(*a, **kw): 
Too many open files  File "zmq/backend/cython/socket.pyx", line 330, in zmq.backend.cython.socket.Socket.__init__

zmq.error.ZMQError: Too many open files
Exception in thread Thread-1276:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-7d95b7becf25>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration
Exception in th

replaying at 1238 xgboost
replaying at 1239 xgboost
replaying at 1240 xgboost
replaying at 1241 xgboost
replaying at 1242 xgboost
replaying at 1243 xgboost
replaying at 1244 xgboost
replaying at 1245 xgboost
replaying at 1246 xgboost



Exception in thread Thread-1273:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-7d95b7becf25>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration



replaying at 1247 xgboost
replaying at 1248 xgboost
replaying at 1249 xgboost
replaying at 1250 xgboost
replaying at 1251 xgboost
replaying at 1252 xgboost
replaying at 1253 xgboost
replaying at 1254 xgboost
replaying at 1255 xgboost
replaying at 1256 xgboost
replaying at 1257 xgboost
replaying at 1258 xgboost
replaying at 1259 xgboost
replaying at 1260 xgboost


In [59]:
data = np.random.rand(100,15) # 5 entities, each contains 10 features
label = np.random.randint(6, size=100)
dtest = xgb.DMatrix([data[0]])
print(label.shape)
DQNAgent().xgbmodel_bld(data,label)
DQNAgent().bst.predict(dtest)
# np.argmax([DQNAgent().bst.predict(dtest),10])

(100,)
[0]	train-merror:0.41	eval-merror:0.41
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0.33	eval-merror:0.33
[2]	train-merror:0.25	eval-merror:0.25
[3]	train-merror:0.25	eval-merror:0.25
[4]	train-merror:0.26	eval-merror:0.26
[5]	train-merror:0.25	eval-merror:0.25
[6]	train-merror:0.19	eval-merror:0.19
[7]	train-merror:0.19	eval-merror:0.19
[8]	train-merror:0.18	eval-merror:0.18
[9]	train-merror:0.16	eval-merror:0.16
[10]	train-merror:0.17	eval-merror:0.17
[11]	train-merror:0.15	eval-merror:0.15
[12]	train-merror:0.15	eval-merror:0.15
[13]	train-merror:0.13	eval-merror:0.13
[14]	train-merror:0.13	eval-merror:0.13
[15]	train-merror:0.13	eval-merror:0.13
[16]	train-merror:0.13	eval-merror:0.13
[17]	train-merror:0.12	eval-merror:0.12
[18]	train-merror:0.13	eval-merror:0.13
[19]	train-merror:0.12	eval-merror:0.12
[20]	train-merror:0.13	eval-merror:0.13
[21]	train-merro

array([ 0.5], dtype=float32)